In [3]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*


import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import xgboost as xgb
# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [4]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [5]:
#create day of week column and hour column and a year colum for both training and test set
train['day']=0
train['hour']=0
train['year']=0
for i in range(len(train)) :
    train['day'][i] = train['date'][i].weekday()
    train['hour'][i] = train['date'][i].hour
    train['year'][i]=train['date'][i].year

test['day']=0
test['hour']=0
test['year']=0
for i in range(len(test)) :
    test['day'][i] = test['date'][i].weekday()
    test['hour'][i] = test['date'][i].hour
    test['year'][i]=test['date'][i].year


In [6]:
#create weekEnd variable 
train['weekEnd'] = 0 ;
train.ix[train['day']>=5,'weekEnd']=1

test['weekEnd'] = 0 ;
test.ix[test['day']>=5,'weekEnd']=1


In [7]:
#create ahot () variable for both training and test set
train['ahot'] = 4
for i in range(len(train)) :
	currentTemp = train['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train['ahot'][i] = 1


test['ahot'] = 4
for i in range(len(test)) :
	currentTemp = test['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test['ahot'][i] = 1



In [8]:
#create ahot () variable for both training and test set
train['hot'] = 4
for i in range(len(train)) :
	currentTemp = train['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train['hot'][i] = 1


test['hot'] = 4
for i in range(len(test)) :
	currentTemp = test['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test['hot'][i] = 2
	elif (currentTemp<13.94) :
		test['hot'][i] = 1



In [9]:
train = train.reset_index()
train = train.drop("datetime",1)
train = train.drop(['date'],1)

test = test.reset_index()
result= test[['datetime']]

test = test.drop("datetime",1)
test = test.drop('date',1)

In [10]:
train_registered=train.drop("casual",1)
train_registered=train_registered.drop("count",1)
train_registered_SEM = train_registered.drop(train_registered[train_registered.weekEnd == 1].index)
train_registered_WE= train_registered.drop(train_registered[train_registered.weekEnd == 0].index)
train_registered_SEM = train_registered_SEM.drop("weekEnd",1)
train_registered_WE = train_registered_WE.drop("weekEnd",1)


train_casual=train.drop("registered",1)
train_casual=train_casual.drop("count",1)
train_casual_SEM = train_casual.drop(train_casual[train_casual.weekEnd == 1].index)
train_casual_WE = train_casual.drop(train_casual[train_casual.weekEnd == 0].index)
train_casual_SEM = train_casual_SEM.drop("weekEnd",1)
train_casual_WE = train_casual_WE.drop("weekEnd",1)


test_WE = test.drop(test[test.weekEnd == 0].index)
test_SEM = test.drop(test[test.weekEnd == 1].index)
test_SEM = test_SEM.drop("weekEnd",1)
test_WE = test_WE.drop("weekEnd",1)

In [20]:
X_train = train_registered_SEM.drop("registered",1)
Y_train = train_registered_SEM['registered']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
params["eval_metric"] = "auc"
params["silent"] = 1
params["max_depth"] = 8
gbm_reg_SEM = xgb.train(dtrain=T_train_xgb, params = params)
#xgb.plot_importance(gbm_reg_SEM)

In [21]:
X_train = train_registered_WE.drop("registered",1)
Y_train = train_registered_WE['registered']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
params["eval_metric"] = "auc"
params["silent"] = 1
params["max_depth"] = 9
gbm_reg_WE = xgb.train(dtrain=T_train_xgb, params = params)
#xgb.plot_importance(gbm_reg_WE)

In [22]:
X_train = train_casual_WE.drop("casual",1)
Y_train = train_casual_WE['casual']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
params["eval_metric"] = "auc"
params["silent"] = 1
params["max_depth"] = 10
gbm_cas_WE = xgb.train(dtrain=T_train_xgb, params = params)
#xgb.plot_importance(gbm_cas_WE)

In [23]:
X_train = train_casual_SEM.drop("casual",1)
Y_train = train_casual_SEM['casual']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
params["eval_metric"] = "auc"
params["silent"] = 1
params["max_depth"] = 10
gbm_cas_SEM = xgb.train(dtrain=T_train_xgb, params = params)
#xgb.plot_importance(gbm_cas_SEM)

In [ ]:
#test_WE.index
plt.show()

In [ ]:
X_test = xgb.DMatrix(test_WE)
Y_pred_WE = gbm_cas_WE.predict(X_test) + gbm_reg_WE.predict(X_test)
print(Y_pred_WE)

X_test = xgb.DMatrix(test_SEM)
Y_pred_SEM = gbm_cas_SEM.predict(X_test) + gbm_reg_SEM.predict(X_test)
print(Y_pred_SEM)

In [ ]:
df_WE = pd.DataFrame(Y_pred_WE)
df_WE.columns=['count']
df_WE['new_index']=test_WE.index
df_WE=df_WE.set_index("new_index")

df_SEM = pd.DataFrame(Y_pred_SEM)
df_SEM.columns=['count']
df_SEM['new_index']=test_SEM.index
df_SEM=df_SEM.set_index("new_index")


In [ ]:
df = df_SEM.append(df_WE)

In [ ]:
result['count'] = df
#result[result['count']<=0] = 0

result_final = result.set_index('datetime')
result_final[result_final['count']<=0]=0
result_final

In [ ]:
result_final.to_csv('Résultats/XGBoost/submissionXGBoost.csv')

In [ ]:
#xgb.cv(params,T_train_xgb)